<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


# Create Player dataset

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

num_of_players = len(df) #250

projections_df = df[['Player', 'Bye', 'POS', 'AVG']].head(num_of_players).copy()
projections_df['Bye'] = projections_df['Bye'].apply(extract_numbers)
projections_df['Bye'] = projections_df['Bye'].fillna(0).astype(int)

projections_df.info()
print(projections_df.value_counts('POS'))
print(projections_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  948 non-null    object 
 1   Bye     948 non-null    int64  
 2   POS     948 non-null    object 
 3   AVG     948 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 29.8+ KB
POS
WR     335
RB     224
TE     166
QB     127
K       64
DST     32
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12      

# Create custom projections
Maybe delete later when real projections available

In [58]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def projection_base(avg, pos, max_val=22, min_val=7, k=75, c=1.5):
    proj = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        proj += 8  # QB-Bonus
    elif pos == "K":
        proj -= 2  # K-Penalty
    elif pos == "DST":
        proj -= 3  # DST-Penalty
    return proj

final_projections = []
for _, row in projections_df.iterrows():
    base_score = projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = projections_df[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]
f["TTL"] = f[weekly_columns].sum(axis=1)
f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))


# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(10)

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,TTL,dropoff
0,Christian McCaffrey,9,RB,1.0,22.978597,22.954585,22.120371,23.597459,20.598261,20.804912,...,19.005679,21.134136,18.600883,24.147962,25.039507,23.273264,25.797615,25.026554,360.148390,6.491459
1,CeeDee Lamb,7,WR,2.6,21.261536,19.554366,23.621496,28.139562,20.551701,25.891952,...,18.534922,25.094996,23.564278,19.200713,23.712556,21.908590,20.741413,19.952138,360.642285,2.836624
2,Tyreek Hill,6,WR,3.2,24.871440,22.156043,20.594444,21.358482,24.120290,0.000000,...,22.804561,17.074552,19.475745,20.099565,20.610620,24.784414,20.489497,24.120291,347.283660,3.305474
3,Bijan Robinson,12,RB,5.0,23.232560,23.179239,21.985265,19.987041,22.061820,21.477777,...,26.315981,21.639443,0.000000,19.308354,18.790444,21.060939,22.391127,21.535163,348.080651,3.723681
4,Breece Hall,12,RB,5.4,21.630176,23.015083,21.714604,22.094791,17.518430,21.788816,...,21.558990,22.657591,0.000000,22.398724,20.611438,24.327525,19.454257,20.571772,334.664215,13.799135
5,Amon-Ra St. Brown,5,WR,6.2,26.491791,21.778246,22.670170,23.696612,0.000000,22.443583,...,25.798878,18.711132,24.041068,18.272806,20.514555,15.759807,19.293837,23.374604,343.978185,2.942266
6,Ja'Marr Chase,12,WR,6.6,22.429309,23.979285,23.755939,20.319740,23.762068,19.667858,...,21.043045,18.862315,0.000000,18.463382,23.021064,17.445810,18.333255,24.130640,337.985258,5.103805
7,Justin Jefferson,6,WR,7.0,19.750330,20.932826,18.227766,19.211842,20.624369,0.000000,...,20.981047,24.335901,23.198947,22.958044,20.531019,18.878749,19.394836,19.877455,332.881453,11.302423
8,Saquon Barkley,5,RB,9.2,21.596898,21.047373,20.033026,25.027654,0.000000,22.314854,...,23.686620,23.534247,21.869948,19.382146,20.268657,20.856557,19.761847,22.843063,353.656931,0.096859
9,A.J. Brown,5,WR,10.2,22.822809,22.734158,19.390678,22.090112,0.000000,18.284024,...,20.303555,21.041346,19.207846,20.662296,19.445979,19.151901,19.295971,19.237688,321.579030,0.060217


In [66]:
# filter for Player == Rico Dowdle
f[f['Player'] == 'Carolina Panthers']

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,TTL,dropoff
408,Carolina Panthers,11,DST,280.0,5.09777,5.899281,6.212631,5.452478,6.109588,5.807687,...,5.935671,0.0,5.242054,6.339039,5.932713,6.463601,6.720042,6.295032,94.595626,0.0


In [75]:
# --- 1. Parameter (aus deinem paste-2.txt) ---
players        = projections_df['Player'].tolist()
positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks          = list(range(1,18))
pos            = dict(zip(projections_df['Player'], projections_df['POS']))
pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {**{int(week.replace("Week_", "")): row[week] for week in week_cols}, 'dropoff': row['dropoff']}
    for _, row in f.iterrows()
}
beta           = {t:140.0 for t in weeks}
gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
alpha, lambda_0, lambda_1, lambda_2, lambda_3 = 1.0, 1, 100, 150, 25
df_sorted = projections_df.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
topk_pct = 0.005
min_pos_req = pos_limit


# --- 2. Teams, DM-Team und Snake-Draft ---
num_teams      = 12
teams          = [f"Team {i+1}" for i in range(num_teams)]
DM_team        = "Team 3"
num_rounds     = 15
draft_order    = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) verbleibende Spieler neu sortieren
    rem  = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) solange Defizite bestehen, aus allen rem dieser Position picken
    if needed:
        # Kandidaten aller benötigten Positionen
        candidates = [p for p in rem if pos[p] in needed]
        # begrenze auf Top-k, falls mehr Kandidaten vorhanden
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: zufällig aus Top-k aller Positionen
    return random.choice(rem[:topk])

# --- 3. Initialisierung ---
rosters        = {tm: [] for tm in teams}
available      = set(players)
draft_log      = []

# --- 4. Hauptschleife über alle Picks ---
for pick_idx, team in enumerate(draft_order, start=1):
    # 4.1 Ranking der verbleibenden Spieler aktualisieren
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
    Rk  = {p: i+1 for i,p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])
    remaining_picks = num_rounds - len(rosters[team])


    # 4.2 DM-Pick via MIP
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # Variablen nur für verbleibende Spieler
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}")
              for i in player_vars}
        x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
             for i in rem for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # Fixiere vergangene Picks
        for p in rosters[team]:
            m += y[p] == 1

        # Objective
        m.objective = (
            lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
          + lambda_1 * xsum(z[t] for t in weeks[:15])
          + lambda_2 * xsum(z[t] for t in weeks[15:])
          + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)
        )

        # Position‐ und Roster‐Constraints
        for j in positions:
            # Müssen mindestens gamma[j] ziehen
            m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
            for t in weeks:
                m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]
        # Roster Anforderungen
        for pos_name, req in min_pos_req.items():
            m.add_constr(
                xsum(y[i] for i in player_vars if pos.get(i) == pos_name) >= req,
                name=f"min_roster_{pos_name}")
        # maximale Picks pro Team
        m += xsum(y[i] for i in rem) <= num_rounds
        # nur gedraftete Spieler in der Week‐Lineup
        for i in rem:
            for t in weeks:
                m += x[i,t] <= y[i]
        # Win‐Indicator
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

        # Robuste Draft‐Constraint (1b) – grob implementiert
        n_k = pick_idx
        for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
            top_cut = int(alpha*(future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i,r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

        m.optimize()
        # gewählten Spieler extrahieren
        chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
    else:
        pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=topk_pct)

    # 4.4 Update
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx, "Team": team, "Player": pick,
        "Round": (pick_idx-1)//len(teams)+1, "POS": pos[pick]
    })

# --- 5. Ergebnis als DataFrame ---
df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1+num_teams*2))


    Pick     Team               Player  Round  POS
0      1   Team 1          CeeDee Lamb      1   WR
1      2   Team 2  Christian McCaffrey      1   RB
2      3   Team 3    Carolina Panthers      1  DST
3      4   Team 4    Amon-Ra St. Brown      1   WR
4      5   Team 5       Bijan Robinson      1   RB
5      6   Team 6          Tyreek Hill      1   WR
6      7   Team 7        Ja'Marr Chase      1   WR
7      8   Team 8           A.J. Brown      1   WR
8      9   Team 9     Justin Jefferson      1   WR
9     10  Team 10      Jonathan Taylor      1   RB
10    11  Team 11          Breece Hall      1   RB
11    12  Team 12       Saquon Barkley      1   RB
12    13  Team 12       Garrett Wilson      2   WR
13    14  Team 11       Kyren Williams      2   RB
14    15  Team 10   Travis Etienne Jr.      2   RB
15    16   Team 9        Derrick Henry      2   RB
16    17   Team 8           Puka Nacua      2   WR
17    18   Team 7         Jahmyr Gibbs      2   RB
18    19   Team 6        Davant

In [9]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

OptimizationStatus.OPTIMAL


In [73]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

POS,DST,K,QB,RB,TE,WR
Team,,,,,,
Team 1,1,1,3,4,1,5
Team 10,1,1,2,5,1,5
Team 11,1,1,2,3,2,6
Team 12,1,1,2,4,3,4
Team 2,1,1,1,4,1,7
Team 3,15,0,0,0,0,0
Team 4,1,1,1,4,2,6
Team 5,1,1,2,4,2,5
Team 6,1,1,2,3,2,6


In [11]:
result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df['Pick Info'] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]  # oder andere gewünschte Spalten

for team, df in result_dfs.items():
    print(f"=== {team} ===")
    print(df)
    print()


=== Team 1 ===
                  Player          Pick Info  POS
0       Baltimore Ravens     Round 1 Pick 1  DST
23        Pat Freiermuth    Round 2 Pick 24   TE
24          Malik Nabers    Round 3 Pick 25   WR
47             Cole Kmet    Round 4 Pick 48   TE
48         Calvin Ridley    Round 5 Pick 49   WR
71           Jordan Love    Round 6 Pick 72   QB
72        Deshaun Watson    Round 7 Pick 73   QB
95        Terry McLaurin    Round 8 Pick 96   WR
96    Brian Robinson Jr.    Round 9 Pick 97   RB
119         Keon Coleman  Round 10 Pick 120   WR
120        Aaron Rodgers  Round 11 Pick 121   QB
143     Brian Thomas Jr.  Round 12 Pick 144   WR
144         Younghoe Koo  Round 13 Pick 145    K
167  Philadelphia Eagles  Round 14 Pick 168  DST
168        Jaylen Wright  Round 15 Pick 169   RB

=== Team 2 ===
              Player          Pick Info  POS
1        Tyreek Hill     Round 1 Pick 2   WR
22   Jonathan Taylor    Round 2 Pick 23   RB
25       Josh Jacobs    Round 3 Pick 26   RB
46   